In [1]:
!kaggle datasets download -d mexwell/telecom-shanghai-dataset

Dataset URL: https://www.kaggle.com/datasets/mexwell/telecom-shanghai-dataset
License(s): other
100%|████████████████████████████████████████| 219M/219M [00:07<00:00, 40.2MB/s]
100%|████████████████████████████████████████| 219M/219M [00:07<00:00, 31.4MB/s]


In [2]:
!unzip /kaggle/working/telecom-shanghai-dataset.zip

Archive:  /kaggle/working/telecom-shanghai-dataset.zip
  inflating: data_10.110.15.xlsx     
  inflating: data_10.1610.31.xlsx    
  inflating: data_11.111.15.xlsx     
  inflating: data_11.1611.30.xlsx    
  inflating: data_6.16.15.xlsx       
  inflating: data_6.166.30.xlsx      
  inflating: data_7.167.31.xlsx      
  inflating: data_7.17.15.xlsx       
  inflating: data_8.168.31.xlsx      
  inflating: data_8.18.15.xlsx       
  inflating: data_9.169.30.xlsx      
  inflating: data_9.19.15.xlsx       
